<a href="https://colab.research.google.com/github/Seoyunki/Project_01/blob/main/N422a_Distributed_Representation_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [3]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [4]:
import gensim

gensim.__version__

'4.2.0'

In [5]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### 1. Word2Vec과 코사인 유사도

word2vec을 이용해 구한 'data'와 'science'임베딩 값의 코사인 유사도를 구해보겠습니다. sklearn의 cosine_similarity를 이용하겠습니다.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

result = cosine_similarity([wv['data'], wv['science']])

print(f'{result[1][0]:.4f}')

0.1576


문항 1) 위에서 구한 코사인 유사도를 소수점 3째자리까지 입력하세요

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 2. 텍스트 분류

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [8]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [9]:
df = pd.read_csv("spam.csv", encoding='latin-1')

In [10]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [11]:
df = df[['v1', 'v2']]

In [12]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df.v1.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [14]:
encoder = LabelEncoder()
y = encoder.fit_transform(df.v1)

In [15]:
X = df['v2']

In [16]:
X.shape

(5572,)

In [17]:
y.shape

(5572,)

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y )

In [20]:
X_train.shape

(4736,)

In [21]:
X_test.shape

(836,)

In [22]:
tokenizer = Tokenizer(num_words=1000)

In [23]:
tokenizer.fit_on_texts(X_train)

In [24]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8186

In [25]:
X_train_encoded = tokenizer.texts_to_sequences(X_train)

In [26]:
max_len = max(len(sent) for sent in X_train_encoded)
max_len

172

In [27]:
sent_lengths = [len(sent) for sent in X_train_encoded]
np.mean(sent_lengths)

12.666173986486486

In [28]:
X_train_pad = pad_sequences(X_train_encoded, maxlen=150, padding='post')
X_train_pad.shape

(4736, 150)

In [29]:
embedding_matrix = np.zeros((vocab_size, 300))
embedding_matrix.shape

(8186, 300)

In [30]:
def get_vector(word):
  if word in wv:
    return wv[word]
  else:
    return None
  
for word, i in tokenizer.word_index.items():
  temp = get_vector(word)
  if temp is not None:
    embedding_matrix[i] = temp

In [31]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=150, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train_pad, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
60/60 [==============================] - 2s 12ms/step - loss: 0.6692 - acc: 0.8685 - val_loss: 0.6513 - val_acc: 0.8544
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6330 - acc: 0.8688 - val_loss: 0.6192 - val_acc: 0.8544
Epoch 3/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6019 - acc: 0.8688 - val_loss: 0.5920 - val_acc: 0.8544
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5752 - acc: 0.8688 - val_loss: 0.5685 - val_acc: 0.8544
Epoch 5/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5522 - acc: 0.8688 - val_loss: 0.5482 - val_acc: 0.8544
Epoch 6/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5322 - acc: 0.8688 - val_loss: 0.5310 - val_acc: 0.8544
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5151 - acc: 0.8688 - val_loss: 0.5160 - val_acc: 0.8544
Epoch 8/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5002 - acc: 0.8

In [33]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_encoded, maxlen=150, padding='post')

In [34]:
model.evaluate(X_test_pad, y_test)

27/27 [==============================] - 0s 3ms/step - loss: 0.4698 - acc: 0.8660


[0.4698384404182434, 0.8660287261009216]

### 3)Word2Vec에서의 OOV 문제

```
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
```
Lecture Note에 있는 위의 코드를 변형하여, OOV 개수를 확인해주세요.
- tokenizer는 위에서 활용한 tokenizer를 그대로 사용하겠습니다.
- Tip : dictionary를 활용하거나, Counter를 활용해보세요.

In [57]:
### 이곳에서 과제를 수행해 주세요 ###

def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X)

oov = {}
keys = []
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is None:
      if word not in keys:
        oov[word] = 1
        keys.append(word)
      else:
        oov[word] = oov[word] + 1
    

print(len(oov))

2663


In [58]:
oov

{'to': 1,
 'a': 1,
 'and': 1,
 'of': 1,
 "i'm": 1,
 "i'll": 1,
 '150p': 1,
 "i've": 1,
 '50': 1,
 '16': 1,
 'ìï': 1,
 '18': 1,
 'thk': 1,
 'å£1': 1,
 'å£1000': 1,
 '150ppm': 1,
 '10': 1,
 'didnt': 1,
 '000': 1,
 'tmr': 1,
 'å£100': 1,
 '500': 1,
 '10p': 1,
 'colour': 1,
 'å£5000': 1,
 "''": 1,
 '8007': 1,
 'å£2000': 1,
 'chikku': 1,
 'å£500': 1,
 '86688': 1,
 "c's": 1,
 '750': 1,
 '11': 1,
 '12hrs': 1,
 '100': 1,
 'pobox': 1,
 'smth': 1,
 '08000930705': 1,
 'mobileupd8': 1,
 'goodmorning': 1,
 'oredi': 1,
 'suite342': 1,
 '2lands': 1,
 '08000839402': 1,
 'å£3': 1,
 '\x89û': 1,
 '30': 1,
 'freemsg': 1,
 "'": 1,
 'frnds': 1,
 'ni8': 1,
 'å£250': 1,
 'savamob': 1,
 '0800': 1,
 '03': 1,
 'mrng': 1,
 '87066': 1,
 'bslvyl': 1,
 'å£2': 1,
 '2003': 1,
 '800': 1,
 'å£350': 1,
 "i'd": 1,
 '20': 1,
 'askd': 1,
 'call2optout': 1,
 '04': 1,
 '12': 1,
 'abiola': 1,
 'rply': 1,
 'muz': 1,
 'w1j6hl': 1,
 'callertune': 1,
 'custcare': 1,
 'rakhesh': 1,
 'ldn': 1,
 '00': 1,
 'å£10': 1,
 'getzed': 1,
 'n